In [ ]:
#pip install -U spacy
#pip install --user en_core_web_sm-3.0.0.tar.gz

In [1]:
#with open("sample.txt", "r") as fh:
#with open("hdfs-audit.log", "r") as fh:
with open("sample1.txt", "r") as fh:
    fstring = fh.readlines()
print(len(fstring))

61


In [3]:
import re

file_lst=[]
with open("sample1.txt","r") as fh:
    for line in fh:
        res = re.compile(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})')
        ent_tup=()
        ent_lst=[]
        file_tup=[]
        ent_dict={"entities":[]}
        for m in res.finditer(line):
            ent_tup = (m.start(),m.end(),"IP")
            ent_lst.append(ent_tup)
        ent_dict["entities"] = ent_lst
        file_tup = (line,ent_dict)
        file_lst.append(file_tup)        
        #print(file_tup)
print(len(file_lst))
print(file_lst)

61
[('2021-04-21 00:00:00,034 INFO 10.45.149.37.audit: allowed=true\tugi=hdfs (auth:SIMPLE)\tip=/10.45.149.67\tcmd=getfileinfo\tsrc=/user\tdst=null\tperm=null\tproto=rpc\n', {'entities': [(29, 41, 'IP'), (89, 101, 'IP')]}), ('2021-04-21 00:00:00,036 INFO FSNamesystem.audit: allowed=true\tugi=hdfs (auth:SIMPLE)\tip=/10.45.149.67\tcmd=listStatus\tsrc=/user\tdst=null\tperm=null\tproto=rpc\n', {'entities': [(89, 101, 'IP')]}), ('2021-04-21 00:00:00,038 INFO FSNamesystem.audit: allowed=true\tugi=hdfs (auth:SIMPLE)\tip=/10.45.149.67\tcmd=getfileinfo\tsrc=/user/ambari-qa/.Trash\tdst=null\tperm=null\tproto=rpc\n', {'entities': [(89, 101, 'IP')]}), ('2021-04-21 00:00:00,040 INFO FSNamesystem.audit: allowed=true\tugi=hdfs (auth:SIMPLE)\tip=/10.45.149.67\tcmd=getfileinfo\tsrc=/user/hbase/.Trash\tdst=null\tperm=null\tproto=rpc\n', {'entities': [(89, 101, 'IP')]}), ('2021-04-21 00:00:00,042 INFO FSNamesystem.audit: allowed=true\tugi=hdfs (auth:SIMPLE)\tip=/10.45.149.67\tcmd=getfileinfo\tsrc=/user/h

In [4]:
import spacy
import en_core_web_sm as en
print(spacy.__version__)
print(en.__version__)

3.0.5
3.0.0


In [ ]:
#from spacy.language import Language
#@Language.component("info_component")
#def my_component(doc):
#    print(f"After tokenization, this doc has {len(doc)} tokens.")
#    print("The part-of-speech tags are:", [token.pos_ for token in doc])
#    if len(doc) < 10:
#        print("This is a pretty short document.")
#    return doc

In [ ]:
#source_nlp = spacy.load("en_core_web_sm")
#print(source_nlp.pipe_names)
##mer = source_nlp.get_pipe("ner")
##print(mer)
#nlp = spacy.blank("en")
#nlp.add_pipe("ner",source=source_nlp)
#print(nlp.pipe_names)

In [ ]:
#Add new pipe - "print_info" into nlp object
#nlp.add_pipe("info_component", name="print_info", first=True)
#print(nlp.pipe_names)  # ['tagger', 'parser', 'ner', 'print_info']
#doc = nlp("This is a sentence.")

In [5]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Australia")
for ent in doc.ents:
    print(ent.text,ent.start_char,ent.end_char,ent.label_)

Australia 0 9 GPE


In [6]:
nlp.pipe_names
ner = nlp.get_pipe("ner")
print(ner)

In [7]:
for _, annotations in file_lst:
    for ent in annotations.get("entities"):
        #print(ent[2])
        ner.add_label(ent[2])

In [8]:
disable_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
print(disable_pipes)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']


In [15]:
import warnings
warnings.filterwarnings("ignore")
import random
from spacy.util import minibatch, compounding
from spacy.training import Example

with nlp.disable_pipes(*disable_pipes):
    
    optimizer = nlp.resume_training()
    
    for iteration in range(50):
        
        random.shuffle(file_lst)
        losses={}
        
        for batch in minibatch(file_lst, size=5):
            for text,annotation in batch:
                doc = nlp.make_doc(text)
                #print("text",text)
                #print("annot",annotation)
                #print("doc",doc)
                example = Example.from_dict(doc,annotation)
                #print(example)
                ner.update([example],losses=losses,drop=0.3,sgd=optimizer)
        print(f'Loss in iter {iteration} is {losses}')

Loss in iter 0 is {'ner': 1.9007353300574082e-06}
Loss in iter 1 is {'ner': 8.412306400839159e-10}
Loss in iter 2 is {'ner': 1.3888409166285397e-09}
Loss in iter 3 is {'ner': 2.8042107557659804e-07}
Loss in iter 4 is {'ner': 1.074512990049028e-08}
Loss in iter 5 is {'ner': 2.888840762146031e-07}
Loss in iter 6 is {'ner': 4.547451041261041e-09}
Loss in iter 7 is {'ner': 4.9620355048697804e-08}
Loss in iter 8 is {'ner': 1.544050916507232e-09}
Loss in iter 9 is {'ner': 1.3013193861910948e-08}
Loss in iter 10 is {'ner': 5.56209181982719e-10}
Loss in iter 11 is {'ner': 1.3815435840495057e-09}
Loss in iter 12 is {'ner': 8.541957820985273e-10}
Loss in iter 13 is {'ner': 7.540852419833791e-10}
Loss in iter 14 is {'ner': 2.19224839581073e-09}
Loss in iter 15 is {'ner': 6.246685173199007e-09}
Loss in iter 16 is {'ner': 1.0142467760558312e-08}
Loss in iter 17 is {'ner': 1.0032383725475044e-10}
Loss in iter 18 is {'ner': 4.726199884318611e-10}
Loss in iter 19 is {'ner': 7.31372653946788e-11}
Loss 

In [21]:
for text, _ in file_lst:
    #print(text)
    doc=nlp(text)
    print("Entities",[(ent.text,ent.label_) for ent in doc.ents])

Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []
Entities []


In [19]:
from spacy import displacy
doc = nlp("the IP which is causing issue in 192.167.23.23")
for ent in doc.ents:
    print(ent)
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
displacy.render(nlp(doc.text),style='ent',jupyter=True)